In [2]:
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

from datetime import datetime, timedelta

import operator

import mpmath as mp
from importlib import reload

from utils import utils, net_lstm, net_lstm_w2v
from tqdm import tqdm
import random
sns.set(style="whitegrid")


torch.manual_seed(1)

In [3]:
reload(utils)
# Getting a list of valid candidates given properties
pep_info = pd.read_pickle("data_inf.pkl")

pep_600 = list(pep_info[(pep_info["length"]>1000)].index)
good_peps = np.loadtxt('good_peps.txt',dtype=int)

# Defining users for the different data sets.
train_peps = good_peps[good_peps<=600]
train_peps = np.array(list(set(train_peps).intersection(pep_600)))

train_peps_all = good_peps
train_peps_all = np.array(list(set(train_peps_all).intersection(pep_600)))

train_peps_vali = good_peps[good_peps>600]
train_peps_vali = np.array(list(set(train_peps_vali).intersection(pep_600)))

# Load in data of the test set
_, dat_test_vali = utils.load_train_test_set(train_peps_vali,bs=10)

In [4]:
# Create network
reload(net_lstm_w2v)
net = net_lstm_w2v.Brain()
net.bs = 1
net.seq_len=100

net.create_model()

In [5]:
# Load in previously trained weights
net.model.load_state_dict(torch.load("Network/network_epoch_0.338_Feb-23-2020"))

<All keys matched successfully>

In [6]:
# Get predictions of the network
net_res = net.test(dat_test_vali,pep_acc = True)

Test loss: 1.161 | Test accu: 0.340 | Test accu expl: 0.814 | Test expl AUC: 0.708 | 

In [7]:
# Defining how to plot on a map
import geoplotlib
from geoplotlib.utils import BoundingBox
from geoplotlib.layers import BaseLayer
from geoplotlib.core import BatchPainter
    
class CustomLayer(BaseLayer):

    def __init__(self, data_true, data_guess):
        self.data_true = data_true
        self.data_guess = data_guess
        self.painter = BatchPainter()

    def invalidate(self, proj):
        x_true, y_true = proj.lonlat_to_screen(self.data_true['lon'], self.data_true['lat'])
        x_guess, y_guess = proj.lonlat_to_screen(self.data_guess['lon'], self.data_guess['lat'])
        #Previous true
        self.painter.set_color('red') #blue
        self.painter.lines(x_true[:-2], y_true[:-2], x_true[1:-1], y_true[1:-1] ,width = 3)
        self.painter.points(x_true[:-1], y_true[:-1],8,True)
        self.painter.set_color('black')
        self.painter.points(x_true[-2], y_true[-2],8,True)
        if self.data_true['lon'][-1] == self.data_true['lat'][-1]:
            #Next guess
            self.painter.set_color([0,255,0,255]) #blue
            self.painter.points(x_guess[-1], y_guess[-1],8,True)
            self.painter.lines(x_true[-2], y_true[-2], x_guess[-1], y_guess[-1] ,width = 3)
        else:
            #Next guess
            self.painter.set_color([0,0,255,255]) #blue
            self.painter.points(x_guess[-1], y_guess[-1],8,True)
            self.painter.set_color([0,0,255,100]) #blue
            self.painter.lines(x_true[-2], y_true[-2], x_guess[-1], y_guess[-1] ,width = 3)
            #Next true
            self.painter.set_color([255,0,255,255]) #purple
            self.painter.points(x_true[-1], y_true[-1],8,True)
            self.painter.set_color([255,0,255,100]) #purple
            self.painter.lines(x_true[-2], y_true[-2], x_true[-1], y_true[-1] ,width = 3)

    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        self.painter.batch_draw()



In [8]:
# Plotting 6 maps of the trajectory
pep = 718
path_guess_target = np.transpose(np.array([net_res[1][pep]['guess'],net_res[1][pep]['target']]))
with open(f"Data_m_loc/{pep}/stop_coords.json") as json_file:
    data_loc = np.array(json.load(json_file))
df_labels = pd.read_pickle(f"Data/{pep}/label_dict.pkl")
with open(f"Data_m_loc/{pep}/final_series.json") as json_file:
    data_path = np.array(json.load(json_file))
data_path = np.array([i[-1] for i in data_path])
true_labels = data_path[int(data_path.shape[0]*0.9):]
guess_labels = (df_labels.loc[net_res[1][pep]['guess']]).to_numpy().ravel().astype('int')

ii_start = 17  #51
for j in range(2,8):
    ii_end = ii_start+j
    dict_lat_lon_true = {'lat':data_loc[true_labels[ii_start:ii_end]].transpose()[0],'lon':data_loc[true_labels[ii_start:ii_end]].transpose()[1]}
    dict_lat_lon_guess = {'lat':data_loc[guess_labels[ii_start:ii_end]].transpose()[0],'lon':data_loc[guess_labels[ii_start:ii_end]].transpose()[1]}
    data_guess = dict_lat_lon_guess
    data_true = dict_lat_lon_true
    geoplotlib.add_layer(CustomLayer(data_true,data_guess))
    geoplotlib.set_bbox(BoundingBox.from_points(data_true["lon"],data_true["lat"]).KBH)
    #geoplotlib.savefig(f"figures/gps_move_plot_{j}")
    geoplotlib.show()